<a href="https://colab.research.google.com/github/saurav714/Simple-python-practice/blob/main/simple_llm_using_dataset_to_predict_which_type_of_dieses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
pip install safetensors


In [4]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    offload_folder="./offload"
)


The repository OpenAssistant/falcon-7b-sft-top1 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/OpenAssistant/falcon-7b-sft-top1 .
 You can inspect the repository content at https://hf.co/OpenAssistant/falcon-7b-sft-top1.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


RWForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

RWForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
RWForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4

In [20]:
# 🚀 STEP 1: Install required packages
!pip install -qU openai kagglehub pandas

# ✅ STEP 2: Set NVIDIA API Key for NIM endpoint
import os
import getpass

os.environ["NVIDIA_API_KEY"] = getpass.getpass("🔐 Enter your NVIDIA API Key:")

# ✅ STEP 3: Load NVIDIA-hosted GPT-OSS-20B via OpenAI SDK
from openai import OpenAI

client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=os.environ["NVIDIA_API_KEY"]
)


🔐 Enter your NVIDIA API Key:··········


In [11]:
# ✅ STEP 4: Download disease-symptom dataset from Kaggle
import kagglehub

path = kagglehub.dataset_download("itachi9604/disease-symptom-description-dataset")
print("✅ Dataset downloaded to:", path)


✅ Dataset downloaded to: /kaggle/input/disease-symptom-description-dataset


In [15]:
# STEP 1: Install kagglehub if not already
!pip install -q kagglehub

# STEP 2: Download the dataset from Kaggle
import kagglehub

# Download and get the path to the dataset
path = kagglehub.dataset_download("itachi9604/disease-symptom-description-dataset")
print("✅ Dataset downloaded to:", path)

# STEP 3: Load the CSV files using the correct filenames
import pandas as pd
import os

# Check files in the dataset path
print("📂 Files in dataset folder:", os.listdir(path))

# Read the CSV files (verify filenames if needed)
df_dataset = pd.read_csv(os.path.join(path, "dataset.csv"))
df_descriptions = pd.read_csv(os.path.join(path, "symptom_Description.csv"))




✅ Dataset downloaded to: /kaggle/input/disease-symptom-description-dataset
📂 Files in dataset folder: ['symptom_Description.csv', 'Symptom-severity.csv', 'symptom_precaution.csv', 'dataset.csv']


In [22]:
# 🚀 STEP 1: Install necessary packages
!pip install -qU openai kagglehub pandas

# 🚀 STEP 2: Set your NVIDIA API Key securely
import os
import getpass

# Use getpass to securely input the NVIDIA API Key
os.environ["NVIDIA_API_KEY"] = getpass.getpass("🔐 Enter your NVIDIA API Key:")

# 🚀 STEP 3: Initialize OpenAI client for NVIDIA GPT-OSS-20B
from openai import OpenAI

client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=os.environ["NVIDIA_API_KEY"]
)

# 🚀 STEP 4: Download dataset from Kaggle using kagglehub
import kagglehub

# This will download the dataset to ~/.kagglehub/datasets/itachi9604/disease-symptom-description-dataset/
dataset_path = kagglehub.dataset_download("itachi9604/disease-symptom-description-dataset")
print("✅ Dataset downloaded to:", dataset_path)

# OPTIONAL: Explore the dataset
import pandas as pd
import os

# Get CSV files in the dataset path
csv_files = [f for f in os.listdir(dataset_path) if f.endswith('.csv')]

# Load the first CSV file (you can change this if needed)
df = pd.read_csv(os.path.join(dataset_path, csv_files[0]))
print("✅ Loaded CSV:", csv_files[0])
print(df.head())


🔐 Enter your NVIDIA API Key:··········
✅ Dataset downloaded to: /kaggle/input/disease-symptom-description-dataset
✅ Loaded CSV: symptom_Description.csv
          Disease                                        Description
0   Drug Reaction  An adverse drug reaction (ADR) is an injury ca...
1         Malaria  An infectious disease caused by protozoan para...
2         Allergy  An allergy is an immune system response to a f...
3  Hypothyroidism  Hypothyroidism, also called underactive thyroi...
4       Psoriasis  Psoriasis is a common skin disorder that forms...


In [25]:
def ask_gpt(prompt, model="gpt-oss-20b", max_tokens=300, temp=0.7):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful medical assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=temp,
            max_tokens=max_tokens,
        )
        return response.choices[0].message.content
    except Exception as e:
        print("❌ Error:", e)
        return None


In [26]:
response = ask_gpt("""
The patient is experiencing the following symptoms:
- muscle_pain
- fatigue
- high_fever
- skin_rash

Which disease might this indicate? Explain why.
""")
print("🧠 GPT-OSS-20B Output:\n", response)


❌ Error: 404 page not found
🧠 GPT-OSS-20B Output:
 None
